In [13]:
library(tidyverse)
library(readr)
library(GenomicFeatures)
library(DESeq2)
library(org.Mm.eg.db)
library(rjson)
library(tximport)
library(DBI)
library(rje)
library(plyr)

codedir <- getwd()
outdir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_RNA/0_salmon_DE_out/2_DEseq/ENSEMBL_ID'
gndir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_RNA/0_salmon_DE_out/2_DEseq/gene_name'

In [3]:
######################################## Convert ENSEMBL ID to gene symbols ########################################
# Download convert table from:http://useast.ensembl.org/biomart/martview/8c1957c27101a044a318d51140a289e1

cv_file <- '/home/pipkin/references/mm_BioMart_GeneStableID_GeneName.txt'
cv_tb <- read_csv(cv_file)

matchGN <- function(input, outfilename, cvTb=cv_tb){
    colnames(input) <- c("ensembl_stable_ID", colnames(input)[2:length(colnames(input))])
    output <- cvTb %>% right_join(input, by="ensembl_stable_ID")
    output$ensembl_stable_ID <- NULL
    write_csv(output, outfilename)
}


── Column specification ─────────────────────────────────────────────────────────────────────────────────────
cols(
  ensembl_stable_ID = col_character(),
  gene_name = col_character()
)




## 0. Prepare reference

In [4]:
###--- Make reference
#txdb <- makeTxDbFromGFF('/home/pipkin/references/GRCm38.99/Mus_musculus.GRCm38.99.gtf')
#saveDb(txdb, file='/home/pipkin/references/GRCm38.99/Mus_musculus.GRCm38.99')
mmRef <- '/home/pipkin/references/GRCm38.102/Mus_musculus.GRCm38.102'

###--- Convert transcript ID to gene ID
txdb <- loadDb(mmRef)
k <- keys(txdb, "GENEID")
res <- AnnotationDbi::select(txdb, k, "TXNAME", "GENEID")
tx2gene <- res[,2:1]

'select()' returned 1:many mapping between keys and columns



In [6]:
##########---------- Read Quant Files
colData <- read.csv('/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_RNA/Info/meta.csv')
files <- file.path("/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_RNA/0_salmon_DE_out/1_salmon_nonTrim",
                   colData$Name,"quant.sf")
names(files) <- colData$Cond
txi <- tximport(files, type="salmon", tx2gene=tx2gene, ignoreTxVersion = TRUE, dropInfReps = TRUE) # Drop in freps TURE = ignore verison  # Ignore TX verison stringsplits on . 

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 


transcripts missing from tx2gene: 1499

summarizing abundance

summarizing counts

summarizing length



In [9]:
# Construct sampleTable
sampleTable <- data.frame(condition =factor(colData$Type))
rownames(sampleTable) <- colnames(txi$counts)

#import into DESEQ2 framework
dds <- DESeqDataSetFromTximport(txi, sampleTable, ~ condition)
summary(dds)


dds <- DESeq(dds) #RunDESEQ

using counts and average transcript lengths from tximport



[1] "DESeqDataSet object of length 34124 with 0 metadata columns"

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [12]:
results <- as.data.frame(results(dds, contrast = c("condition","WT_MP","WT_TE"))) # makes the fold change 1st elment is the numerator 2nd denominator 
write.csv(results,file.path(outdir,"WT_MP-WT_TE-Salmon.csv"))

In [16]:
for (de_out in list.files(path=outdir, pattern='.csv')){
    de_out_file <- file.path(outdir, de_out)
    new_out_file <- file.path(gndir, gsub(".csv", "_gn.csv", de_out))
    matchGN(read_csv(de_out_file), new_out_file)
}

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ─────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ─────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ─────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ─────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ─────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ─────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(